In [1]:
import os
import zipfile
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
import tensorflow as tf

In [2]:
# current_dir = "dogs_and_cats"
# with zipfile.ZipFile(f"../{current_dir}/train.zip","r") as z:
#     z.extractall(".")
    
# with zipfile.ZipFile(f"../{current_dir}/test.zip","r") as z:
#     z.extractall(".")

In [3]:
main_dir = os.getcwd()
# train_dir = "train/"
# path = os.path.join(main_dir,train_dir)

# for p in os.listdir(path):
#     category = p.split(".")[0]
#     img_array = cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)
#     new_img_array = cv2.resize(img_array, dsize=(80, 80))
#     plt.imshow(new_img_array,cmap="gray")
#     break

In [4]:
# X = []
# y = []
# convert = lambda category: int(category == 'dog')

# def create_test_data(path):
#     for p in os.listdir(path):
#         category = p.split(".")[0]
#         category = convert(category)
        
#         img_array = cv2.imread(os.path.join(path,p), cv2.IMREAD_GRAYSCALE)
#         new_img_array = cv2.resize(img_array, dsize=(80, 80))
        
#         X.append(new_img_array)
#         y.append(category)

In [5]:
# create_test_data(path)

In [6]:
# X = np.array(X).reshape(-1, 80, 80, 1)
# y = np.array(y)

In [7]:
# X.shape, y.shape

In [8]:
# pickle.dump(X, open('process_data/train_x.pkl', 'wb'))
# pickle.dump(y, open('process_data/train_y.pkl', 'wb'))

In [9]:
X = pickle.load(open('pickle_data/train_x.pkl', 'rb'))
y = pickle.load(open('pickle_data/train_y.pkl', 'rb'))

In [10]:
X.shape, y.shape

((25000, 80, 80, 1), (25000,))

In [11]:
#Normalize data
X = X/255.0

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10


In [ ]:
test_dir = "test1"
path = os.path.join(main_dir,test_dir)

X_test = []
id_line = []
def create_test1_data(path):
    for p in os.listdir(path):
        id_line.append(p.split(".")[0])
        img_array = cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)
        new_img_array = cv2.resize(img_array, dsize=(80, 80))
        X_test.append(new_img_array)
create_test1_data(path)
X_test = np.array(X_test).reshape(-1,80,80,1)
X_test = X_test/255

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predicted_val = [int(round(p[0])) for p in predictions]